Простой пример: Защитный Заместитель (Protection Proxy)
Представим доступ к секретному хранилищу. Мы хотим разрешить операцию чтения/записи только пользователям с правильным паролем. Заместитель будет проверять пароль перед обращением к реальному хранилищу.

In [1]:
from abc import ABC, abstractmethod

# 1. Subject Interface (Общий интерфейс)
class VaultAccess(ABC):
    """Интерфейс для доступа к хранилищу."""
    @abstractmethod
    def get_secret_data(self) -> str:
        pass

    @abstractmethod
    def set_secret_data(self, data: str) -> None:
        pass

# 2. Real Subject (Реальный Субъект)
class SecretVault(VaultAccess):
    """Реальное хранилище, выполняющее операции."""
    def __init__(self):
        print("SecretVault: Initializing the real vault (could be resource intensive).")
        self._secret_data = "Initial Secret Information"

    def get_secret_data(self) -> str:
        print("SecretVault: Providing secret data.")
        return self._secret_data

    def set_secret_data(self, data: str) -> None:
        print(f"SecretVault: Setting secret data to '{data}'.")
        self._secret_data = data

# 3. Proxy (Заместитель)
class SecureVaultProxy(VaultAccess):
    """
    Защитный Заместитель: контролирует доступ к SecretVault.
    Требует пароль для выполнения операций.
    """
    def __init__(self, real_vault: SecretVault, password: str):
        print("SecureVaultProxy: Initializing proxy.")
        self._real_vault = real_vault
        self._required_password = "correct_password"
        self._provided_password = password
        self._authenticated = False

    def _authenticate(self) -> bool:
        """Проверяет, аутентифицирован ли пользователь."""
        if not self._authenticated:
            print("SecureVaultProxy: Checking credentials...")
            if self._provided_password == self._required_password:
                print("SecureVaultProxy: Authentication successful!")
                self._authenticated = True
            else:
                print("SecureVaultProxy: Authentication failed! Access denied.")
                self._authenticated = False
        return self._authenticated

    def get_secret_data(self) -> str:
        print("SecureVaultProxy: Attempting to get data...")
        if self._authenticate():
            # Делегируем вызов реальному объекту
            return self._real_vault.get_secret_data()
        else:
            return "ACCESS DENIED: Incorrect password."

    def set_secret_data(self, data: str) -> None:
        print("SecureVaultProxy: Attempting to set data...")
        if self._authenticate():
            # Делегируем вызов реальному объекту
            self._real_vault.set_secret_data(data)
        else:
            print("ACCESS DENIED: Cannot set data. Incorrect password.")

# 4. Client Code (Клиентский Код)
if __name__ == "__main__":
    # Реальный объект создается сразу (может быть скрыто за фабрикой)
    real_vault = SecretVault()

    print("\n--- Accessing through Proxy with CORRECT password ---")
    correct_proxy = SecureVaultProxy(real_vault, "correct_password")
    print("Data:", correct_proxy.get_secret_data())
    correct_proxy.set_secret_data("Updated Secret!")
    print("Data after update:", correct_proxy.get_secret_data()) # Повторная аутентификация не нужна

    print("\n--- Accessing through Proxy with INCORRECT password ---")
    incorrect_proxy = SecureVaultProxy(real_vault, "wrong_password")
    print("Data:", incorrect_proxy.get_secret_data())
    incorrect_proxy.set_secret_data("Trying to hack!") # Запись тоже не пройдет

    # Можно попытаться обратиться напрямую, если есть доступ (но цель Прокси - избежать этого)
    # print("\n--- Direct access (bypassing proxy) ---")
    # print("Direct data:", real_vault.get_secret_data())


# Вывод:
# SecretVault: Initializing the real vault (could be resource intensive).
#
# --- Accessing through Proxy with CORRECT password ---
# SecureVaultProxy: Initializing proxy.
# SecureVaultProxy: Attempting to get data...
# SecureVaultProxy: Checking credentials...
# SecureVaultProxy: Authentication successful!
# SecretVault: Providing secret data.
# Data: Initial Secret Information
# SecureVaultProxy: Attempting to set data...
# SecretVault: Setting secret data to 'Updated Secret!'.
# SecureVaultProxy: Attempting to get data...
# SecretVault: Providing secret data.
# Data after update: Updated Secret!
#
# --- Accessing through Proxy with INCORRECT password ---
# SecureVaultProxy: Initializing proxy.
# SecureVaultProxy: Attempting to get data...
# SecureVaultProxy: Checking credentials...
# SecureVaultProxy: Authentication failed! Access denied.
# Data: ACCESS DENIED: Incorrect password.
# SecureVaultProxy: Attempting to set data...
# ACCESS DENIED: Cannot set data. Incorrect password.

SecretVault: Initializing the real vault (could be resource intensive).

--- Accessing through Proxy with CORRECT password ---
SecureVaultProxy: Initializing proxy.
SecureVaultProxy: Attempting to get data...
SecureVaultProxy: Checking credentials...
SecureVaultProxy: Authentication successful!
SecretVault: Providing secret data.
Data: Initial Secret Information
SecureVaultProxy: Attempting to set data...
SecretVault: Setting secret data to 'Updated Secret!'.
SecureVaultProxy: Attempting to get data...
SecretVault: Providing secret data.
Data after update: Updated Secret!

--- Accessing through Proxy with INCORRECT password ---
SecureVaultProxy: Initializing proxy.
SecureVaultProxy: Attempting to get data...
SecureVaultProxy: Checking credentials...
SecureVaultProxy: Authentication failed! Access denied.
Data: ACCESS DENIED: Incorrect password.
SecureVaultProxy: Attempting to set data...
SecureVaultProxy: Checking credentials...
SecureVaultProxy: Authentication failed! Access denied.
A

Сложный пример: Виртуальный Заместитель (Virtual Proxy) для Загрузки Изображений
Представим приложение, отображающее изображения. Загрузка изображения из файла — "тяжёлая" операция. Виртуальный Заместитель загрузит реальное изображение только тогда, когда оно понадобится для отображения.

In [2]:
from abc import ABC, abstractmethod
import time
import os

# 1. Subject Interface (Общий интерфейс)
class Image(ABC):
    """Интерфейс для работы с изображением."""
    @abstractmethod
    def display(self) -> None:
        """Отображает изображение."""
        pass

    @abstractmethod
    def get_dimensions(self) -> tuple[int, int]:
        """Возвращает размеры изображения (ширина, высота)."""
        pass

# 2. Real Subject (Реальный Субъект)
class RealImage(Image):
    """Реальное изображение, которое загружается из файла."""
    def __init__(self, filename: str):
        self._filename = filename
        self._width = 0
        self._height = 0
        self._image_data = None
        self._load_image() # Загрузка происходит при создании объекта

    def _load_image(self):
        """Имитация долгой загрузки изображения из файла."""
        print(f"RealImage: Loading image '{self._filename}' from disk...")
        # Проверяем, существует ли файл (для имитации)
        if not os.path.exists(self._filename):
             # Создаем пустой файл для имитации
             print(f"RealImage: File '{self._filename}' not found, creating dummy file.")
             with open(self._filename, 'w') as f:
                 f.write("dummy image data")
             time.sleep(0.1) # Небольшая задержка для создания

        # Имитация чтения и обработки
        time.sleep(2) # Значительная задержка!
        with open(self._filename, 'r') as f:
             self._image_data = f.read()
        # Имитация получения размеров
        self._width = 800
        self._height = 600
        print(f"RealImage: Image '{self._filename}' loaded successfully.")

    def display(self) -> None:
        print(f"RealImage: Displaying image '{self._filename}' ({self._width}x{self._height}). Data: '{self._image_data[:20]}...'")

    def get_dimensions(self) -> tuple[int, int]:
        return (self._width, self._height)

# 3. Proxy (Виртуальный Заместитель)
class ImageProxy(Image):
    """
    Виртуальный Заместитель: откладывает создание RealImage до первого обращения.
    Хранит только имя файла до момента реальной загрузки.
    """
    def __init__(self, filename: str):
        print(f"ImageProxy: Initializing proxy for '{filename}' (RealImage NOT loaded yet).")
        self._filename = filename
        self._real_image: RealImage | None = None # Ссылка на реальный объект (пока None)
        self._cached_dimensions: tuple[int, int] | None = None # Можно кэшировать простые данные

    def _ensure_real_image_loaded(self) -> RealImage:
        """Создает RealImage, если он еще не был создан."""
        if self._real_image is None:
            print(f"ImageProxy: First access to '{self._filename}'. Creating RealImage instance...")
            self._real_image = RealImage(self._filename)
        return self._real_image

    def display(self) -> None:
        """Отображение требует загрузки реального изображения."""
        print(f"ImageProxy: Call to display() for '{self._filename}'.")
        real_image = self._ensure_real_image_loaded()
        real_image.display() # Делегируем вызов

    def get_dimensions(self) -> tuple[int, int]:
        """
        Получение размеров может быть оптимизировано.
        Либо загружаем реальное изображение, либо читаем метаданные (здесь кэшируем).
        """
        print(f"ImageProxy: Call to get_dimensions() for '{self._filename}'.")
        if self._cached_dimensions is None:
            if self._real_image is None:
                 # Вариант 1: Загрузить полное изображение (как здесь)
                 real_image = self._ensure_real_image_loaded()
                 self._cached_dimensions = real_image.get_dimensions()
                 # Вариант 2 (лучше): Прочитать только метаданные файла, не грузя все изображение
                 # self._cached_dimensions = self._read_metadata(self._filename)
                 print(f"ImageProxy: Dimensions for '{self._filename}' calculated/fetched and cached.")
            else:
                 self._cached_dimensions = self._real_image.get_dimensions()

        print(f"ImageProxy: Returning cached/fetched dimensions for '{self._filename}'.")
        return self._cached_dimensions

# 4. Client Code
if __name__ == "__main__":
    # Создаем фиктивные файлы для теста
    dummy_files = ["image1.jpg", "image2.png", "image3.gif"]
    for fname in dummy_files:
        if os.path.exists(fname):
            os.remove(fname)

    print("--- Creating Image Proxies ---")
    # Создание прокси происходит быстро, RealImage не создаются
    img1: Image = ImageProxy("image1.jpg")
    img2: Image = ImageProxy("image2.png")
    img3: Image = ImageProxy("image3.gif")

    print("\n--- Accessing Dimensions (might trigger load or use cache) ---")
    # Получение размеров может потребовать загрузки (в нашей реализации)
    dims1 = img1.get_dimensions()
    print(f"Client: Dimensions for image 1: {dims1}")
    # Повторный вызов должен быть быстрым (из кэша прокси)
    dims1_cached = img1.get_dimensions()
    print(f"Client: Dimensions for image 1 (cached): {dims1_cached}")

    dims2 = img2.get_dimensions()
    print(f"Client: Dimensions for image 2: {dims2}")


    print("\n--- Displaying Images (will trigger load if not already loaded) ---")
    # Первый вызов display() для img2 точно вызовет долгую загрузку
    img2.display()
    # Повторный вызов display() для img2 будет быстрым
    print("Displaying img2 again:")
    img2.display()

    # display() для img1 уже был загружен при первом get_dimensions()
    print("Displaying img1:")
    img1.display()

    # img3 еще не загружался
    print("Displaying img3:")
    img3.display()

    # Удаляем созданные фиктивные файлы
    print("\n--- Cleaning up dummy files ---")
    for fname in dummy_files:
        if os.path.exists(fname):
            os.remove(fname)
            print(f"Removed {fname}")


# Примерный вывод:
# --- Creating Image Proxies ---
# ImageProxy: Initializing proxy for 'image1.jpg' (RealImage NOT loaded yet).
# ImageProxy: Initializing proxy for 'image2.png' (RealImage NOT loaded yet).
# ImageProxy: Initializing proxy for 'image3.gif' (RealImage NOT loaded yet).
#
# --- Accessing Dimensions (might trigger load or use cache) ---
# ImageProxy: Call to get_dimensions() for 'image1.jpg'.
# ImageProxy: First access to 'image1.jpg'. Creating RealImage instance...
# RealImage: Loading image 'image1.jpg' from disk...
# RealImage: File 'image1.jpg' not found, creating dummy file.
# RealImage: Image 'image1.jpg' loaded successfully.
# ImageProxy: Dimensions for 'image1.jpg' calculated/fetched and cached.
# ImageProxy: Returning cached/fetched dimensions for 'image1.jpg'.
# Client: Dimensions for image 1: (800, 600)
# ImageProxy: Call to get_dimensions() for 'image1.jpg'.
# ImageProxy: Returning cached/fetched dimensions for 'image1.jpg'.
# Client: Dimensions for image 1 (cached): (800, 600)
# ImageProxy: Call to get_dimensions() for 'image2.png'.
# ImageProxy: First access to 'image2.png'. Creating RealImage instance...
# RealImage: Loading image 'image2.png' from disk...
# RealImage: File 'image2.png' not found, creating dummy file.
# RealImage: Image 'image2.png' loaded successfully.
# ImageProxy: Dimensions for 'image2.png' calculated/fetched and cached.
# ImageProxy: Returning cached/fetched dimensions for 'image2.png'.
# Client: Dimensions for image 2: (800, 600)
#
# --- Displaying Images (will trigger load if not already loaded) ---
# ImageProxy: Call to display() for 'image2.png'.
# RealImage: Displaying image 'image2.png' (800x600). Data: 'dummy image data...'
# Displaying img2 again:
# ImageProxy: Call to display() for 'image2.png'.
# RealImage: Displaying image 'image2.png' (800x600). Data: 'dummy image data...'
# Displaying img1:
# ImageProxy: Call to display() for 'image1.jpg'.
# RealImage: Displaying image 'image1.jpg' (800x600). Data: 'dummy image data...'
# Displaying img3:
# ImageProxy: Call to display() for 'image3.gif'.
# ImageProxy: First access to 'image3.gif'. Creating RealImage instance...
# RealImage: Loading image 'image3.gif' from disk...
# RealImage: File 'image3.gif' not found, creating dummy file.
# RealImage: Image 'image3.gif' loaded successfully.
# RealImage: Displaying image 'image3.gif' (800x600). Data: 'dummy image data...'
#
# --- Cleaning up dummy files ---
# Removed image1.jpg
# Removed image2.png
# Removed image3.gif

--- Creating Image Proxies ---
ImageProxy: Initializing proxy for 'image1.jpg' (RealImage NOT loaded yet).
ImageProxy: Initializing proxy for 'image2.png' (RealImage NOT loaded yet).
ImageProxy: Initializing proxy for 'image3.gif' (RealImage NOT loaded yet).

--- Accessing Dimensions (might trigger load or use cache) ---
ImageProxy: Call to get_dimensions() for 'image1.jpg'.
ImageProxy: First access to 'image1.jpg'. Creating RealImage instance...
RealImage: Loading image 'image1.jpg' from disk...
RealImage: File 'image1.jpg' not found, creating dummy file.
RealImage: Image 'image1.jpg' loaded successfully.
ImageProxy: Dimensions for 'image1.jpg' calculated/fetched and cached.
ImageProxy: Returning cached/fetched dimensions for 'image1.jpg'.
Client: Dimensions for image 1: (800, 600)
ImageProxy: Call to get_dimensions() for 'image1.jpg'.
ImageProxy: Returning cached/fetched dimensions for 'image1.jpg'.
Client: Dimensions for image 1 (cached): (800, 600)
ImageProxy: Call to get_dimension